# Quickstart: Querying PDF With Astra and LangChain

### A question-answering demo using Astra DB and LangChain, powered by Vector Search

#### Pre-requisites:

You need a **_Serverless Cassandra with Vector Search_** database on [Astra DB](https://astra.datastax.com) to run this demo. As outlined in more detail [here](https://docs.datastax.com/en/astra-serverless/docs/vector-search/quickstart.html#_prepare_for_using_your_vector_database), you should get a DB Token with role _Database Administrator_ and copy your Database ID: these connection parameters are needed momentarily.

You also need an [OpenAI API Key](https://cassio.org/start_here/#llm-access) for this demo to work.

#### What you will do:

- Setup: import dependencies, provide secrets, create the LangChain vector store;
- Run a Question-Answering loop retrieving the relevant headlines and having an LLM construct the answer.

Install the required dependencies:

In [2]:
!pip install -q cassio datasets langchain openai tiktoken

In [4]:
! pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


Import the packages you'll need:

In [5]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio

In [8]:
!pip install PyPDF2

In [10]:
from PyPDF2 import PdfReader  # it will hep you to reas the text from pdf

### Setup

In [11]:
# wrong password .. 

ASTRA_DB_APPLICATION_TOKEN = "AstraCS:RhRehNtrHazBdfuT:1390c4ed5a00e51ed12bd2845c748adb29f8462921cc3a2541ec6db" # enter the "AstraCS:..." string found in in your Token JSON file
ASTRA_DB_ID = "8597bed2-0058-4b05--f9cb15b508e4"# enter your Database ID

OPENAI_API_KEY = "sk-proj-3BlbkAt7CYbGJnjactHj6xgtxtZ7vuj1tXuwYRYCN3H25AW8r9xfGiVOdAA" # enter your OpenAI key

#### Provide your secrets:

Replace the following with your Astra DB connection details and your OpenAI API key:

In [12]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/content/apjspeech.pdf')

In [13]:
#Now after reading we are going to divide all content in some chunks

from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [14]:
raw_text

'A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowing important messages: \n \n1. Accelerate development : Aspiration of th

Initialize the connection to your database:

_(do not worry if you see a few warnings, it's just that the drivers are chatty about negotiating protocol versions with the DB.)_

In [ ]:
#  cassio it is the liabrary to intiallize the particular DB

In [15]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

Create the LangChain embedding and LLM objects for later usage:

In [16]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

<ipython-input-16-9c33d76eff6b>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=OPENAI_API_KEY)
<ipython-input-16-9c33d76eff6b>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


Create your LangChain vector store ... backed by Astra DB!

In [17]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

In [18]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [19]:
texts[:50]

['A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at',
 'home makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline

### Load the dataset into the vector store



In [20]:

astra_vector_store.add_texts(texts[:50])  # her we are addind all tet doing embddding and sending it to cassandraDB

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)    # here we are wrapping this all thing inside the wrapper

Inserted 31 headlines.


### Run the QA cycle

Simply run the cells and ask a question -- or `quit` to stop. (you can also stop execution with the "▪" button on the top toolbar)

Here are some suggested questions:
- _What is the current GDP?_
- _How much the agriculture target will be increased to and what the focus will be_


In [ ]:
# Hyderabad are providing one-hour live tele-consultation to doctors in which hospital ?

In [21]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): Hyderabad are providing one-hour live tele-consultation to doctors in which hospital ?

QUESTION: "Hyderabad are providing one-hour live tele-consultation to doctors in which hospital ?"


ANSWER: "Black Lion Hospital"

FIRST DOCUMENTS BY RELEVANCE:


    [0.9173] "tele-medicine in Ethiopia has already been commissioned. Indira Gandh i National Ope ..."
    [0.8973] "internal security forces who are making immense contribution in au gmenting the safe ..."
    [0.8903] "Africa. This was attended by 53 member countries of the Africa n Union, where I 
pro ..."
    [0.8855] "On the evening of February 24, 2007, at Coimbatore, I had a very beautif ul experien ..."

What's your next question (or type 'quit' to exit): quit 


In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): How much the agriculture target will be increased to and what the focus will be

QUESTION: "How much the agriculture target will be increased to and what the focus will be"


ANSWER: "The agriculture credit target will be increased to ` 20 lakh crore with focus on animal husbandry, dairy and fisheries."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9327] "of Millet Research, Hyderabad  will be supported as the Centre of Excellence 
for sh ..."
    [0.9327] "of Millet Research, Hyderabad  will be supported as the Centre of Excellence 
for sh ..."
    [0.9207] "Agriculture Accelerator Fund  
17. An Agriculture Accelerator Fund will be set-up to ..."
    [0.9207] "Agriculture Accelerator Fund  
17. An Agriculture Accelerator Fund will be set-up to ..."

What's your next question (or type 'quit' to exit): What is the current GDP

QUESTION: "What is the current GDP"


ANSWER: "The current GDP is estimated to be 7 per cent."

FIRST DOCUMENTS BY RELEVANCE:


    [0.8920] "estimated to be at 7 per cent. It is notable that this is the highest among all 
the ..."
    [0.8920] "estimated to be at 7 per cent. It is notable that this is the highest among all 
the ..."
    [0.8915] "multiplier impact on growth and employment. After the subdued period of 
the pandemi ..."
    [0.8915] "multiplier impact on growth and employment. After the subdued period of 
the pandemi ..."

What's your next question (or type 'quit' to exit): quit
